<a href="https://colab.research.google.com/github/human-ai2025/Elo-Merchant-Recommendation/blob/master/Preprocessing%20and%20Feature%20Engineering/model_imputation_without_joining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Libraries

In [1]:
#Import Libraries 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import gc
from contextlib import contextmanager
from pandas.core.common import SettingWithCopyWarning
import datetime
import time
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import mode
from sklearn.preprocessing import LabelEncoder

### Mount the drive 

In [2]:
#Mounting drive 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Setup the current path 

In [3]:
!ls

drive  sample_data


In [4]:
#Setting up worksapce directory 
%cd /content/drive/MyDrive/data 

/content/drive/MyDrive/data


In [5]:
!ls

 augmented_test.csv		  LR_cat_2.sav
 augmented_test_try1.csv	  mer_authorized_flag_enc.npy
 augmented_train.csv		  mer_category_1_enc.npy
 augmented_train_try1.csv	  merchant_category_1_enc.npy
 clf_cat2.sav			  merchant_category_4_enc.npy
 clf_cat3.sav			  merchant_most_recent_purchases_range_enc.npy
 clf_name.sav			  merchant_most_recent_sales_range_enc.npy
'Data Dictionary.xlsx'		  merchants.csv
 Data_Dictionary.xlsx		  merged_transactions_with_merchants.csv
 file_name.csv			  new_merchant_authorized_flag_enc.npy
 hist_authorized_flag_enc.npy	  new_merchant_category_1_enc.npy
 hist_category_1_enc.npy	  new_merchant_df.pkl
 hist_category_3_enc.npy	  new_merchant_transactions.csv
 historical_transactions.csv	  sample_submission.csv
 historical_transactions_df.pkl   test.csv
 knn_sal_12.sav			  train.csv
 knn_sal_3.sav			  transactions_raw_merged.csv
 knn_sal_6.sav			  transactions_refined_1_merged_WON.csv


## Timer 

In [6]:
#refer:-https://www.youtube.com/watch?v=vOMtQ4ocMGI
@contextmanager
def timer(title):
    """ used to calculate time for each function"""
    t0 = time.time()
    yield
    print("{} - done in {:.000f}s".format(title, time.time() - t0))

## Memory Reduction 

In [7]:
#https://www.kaggle.com/fabiendaniel/elo-world
#Function to load data into pandas and reduce memory usage

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    #loop for alll the columns in the dataframe 
    for col in df.columns:
        #get the datatype of the column
        col_type = df[col].dtypes
        #if the data type is numeric then only start changing the datatype
        #as it isnt much helpful for other data types 
        if col_type in numerics:
            #stores the min value of the column 
            c_min = df[col].min()
            #stores the maximum value of the column
            c_max = df[col].max()
            #for int type numerics
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            #for float type numerics 
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

## One hot encoder 

In [8]:
def lab_enc(df, cols, prefix=''):
  '''
  categorical to numerical 
  df   - Original DataFrame
  cols - label encode the specified columns
  #https://towardsdatascience.com/choosing-the-right-encoding-method-label-vs-onehot-encoder-a4434493149b#:~:text=Label%20Encoder%3A&text=LabelEncoder%20encode%20labels%20with%20a,value%20to%20as%20assigned%20earlier.&text=The%20categorical%20values%20have%20been,all%20label%20encoding%20is%20about.
  '''
  
  lbl_enc = LabelEncoder()
  for col in cols:
    df[col] = lbl_enc.fit_transform(df[col].astype(str))
    np.save('{}_{}_enc.npy'.format(prefix, col), lbl_enc.classes_)
  return df

## Basic FE on Train Test

In [48]:
def train_test(num_rows=None):

    """
   FUNCTION:
        MERGE THE TRAIN AND TEST AND DOES SOME BASIC FEATURE ENGINEERING 

   ARGS:
        NUM_ROWS: TO LOAD THE NUMBER OF ROWS FOR THE DATAFRAME(DEBUGGING)
        DEFAULT IS NONE
  
   RETURNS:
        THE MERGED DATAFRAME OF TRAIN AND TEST ALONG WITH BASIC FEATURE ENGINEERING 

    """

    # load csv
    train_df = pd.read_csv('/content/drive/MyDrive/data/train.csv', index_col=['card_id'], nrows=num_rows)
    test_df = pd.read_csv('/content/drive/MyDrive/data/test.csv', index_col=['card_id'], nrows=num_rows)

    print("Train samples: {}, test samples: {}".format(len(train_df), len(test_df)))
    OUTLIER_THRESHOLD = 30
    # Create an outliers column set to 1 for
    train_df['outliers'] = np.where(train_df['target'] < OUTLIER_THRESHOLD, 1, 0)

    # set target as nan
    test_df['target'] = np.nan

    # merge
    df = train_df.append(test_df)

    del train_df, test_df
    gc.collect()

    # to datetime
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])

    # datetime features
    df['quarter'] = df['first_active_month'].dt.quarter
    df['elapsed_time'] = (datetime.datetime.today() - df['first_active_month']).dt.days
    df['quarter_first_active_month'] = df['first_active_month'].dt.quarter
    df['first_active_month_diff_from_today'] = (datetime.datetime.today() - df['first_active_month']).dt.days

    for col in ['feature_1', 'feature_2', 'feature_3']:
        order_label = df.groupby(col)['outliers'].mean()
        df[col] = df[col].map(order_label)

    # Some basic statistics transformations over the feature_i columns
    df['feature_sum'] = df['feature_1'] + df['feature_2'] + df['feature_3']
    df['feature_mean'] = df['feature_sum'] / 3
    df['feature_max'] = df[['feature_1', 'feature_2', 'feature_3']].max(axis=1)
    df['feature_min'] = df[['feature_1', 'feature_2', 'feature_3']].min(axis=1)
    df['feature_std'] = df[['feature_1', 'feature_2', 'feature_3']].std(axis=1)

    return df

## Imputations on transactions data 

In [40]:
def transactions_imputations(df_name='hist',nrows = None):

    """
    FUNCTION:
          PERFORMS THE IMPUTATIONS FOR THE TRANSACTIONS DATA 
          IT HANDLES MISSING VALUES 
          IT HANDLES MISSING VALUES BY MODEL IMPUTATIONS 
          IT DOES MAPPING OF CATEGORICAL DATA TO NUMERICAL 

    ARGS:
          DF:- THE DATAFRAME THAT NEEDS TO HAVE IMPUTATIONS 
          DEFAULT IS HISTORICAL TRANSACTIONS 
          NUM_ROWS: TO LOAD THE NUMBER OF ROWS FOR THE DATAFRAME(DEBUGGING)
          DEFAULT IS NONE

    RETURNS:
          THE MODIFIED DATA FRAME WITH IMPUTATIONS 

    REFER:-https://medium.com/towards-artificial-intelligence/handling-missing-data-for-advanced-machine-learning-b6eb89050357
    """

    import pickle
    from sklearn.linear_model import LogisticRegression


    if df_name == 'hist':
      df = pd.read_csv('/content/drive/MyDrive/data/historical_transactions.csv', nrows=nrows)
    else:
      df = pd.read_csv('/content/drive/MyDrive/data/new_merchant_transactions.csv', nrows=nrows)
    

    # reduce memory usage
    df = reduce_mem_usage(df)

    print(df.isna().sum())

    #category 2 and 3 have missing values and we will handle them via model based impuation 

    #creating a dataframe 
    temp = pd.DataFrame()

    #get the card id
    temp['card_id'] = df['card_id']

    #get the merchant id
    temp['merchant_id'] = df['merchant_id']

    #get the purchase date
    temp['purchase_date'] = df['purchase_date']

    #drop the columns from the new merchant data frame 
    df.drop(['card_id', 'merchant_id', 'purchase_date'], axis=1, inplace=True)

    gc.collect()

    #store all the columns as features 
    feat = df.columns

    #having missing values 
    cols = ['category_2', 'category_3']

    #label encode the variables
    df = lab_enc(df, ['authorized_flag','category_1'], prefix=df_name)
    print(df.sample(10))

    #list to hold the null values
    list_no_nan = []

    #select only columns which doesn't have any null values
    for c in feat:
      if c not in cols:
        list_no_nan.append(c)

    #create a test set by selecting only rows which are having null values
    #we will predict the values of categority 2
    test = df[df['category_2'].isna()]

    #create train set by selecting rows which doesn't have any null values
    train = df.dropna()

    d = {'A':1, 'B':2, 'C':3}
    train['category_3'] = train['category_3'].map(d)
    test['category_3'] = test['category_3'].map(d)

    #fit the classifier to the train data
    clf_name = df_name + 'LR1'
    print("[INFO] NAME OF CLF ",clf_name )
    clf_name = LogisticRegression()
    clf_name.fit(train[list_no_nan], train['category_2'])
    #save the model in pickel 
    pickle.dump(clf_name, open('clf_name.sav', 'wb'))

    print("[INFO] Imputing Categorical 2 values ....")
    #make prediction only for the rows with null value
    df.loc[df['category_2'].isna(), 'category_2'] = clf_name.predict(test[list_no_nan])

    test = df[df['category_3'].isna()]
    train = df.dropna()

    clf_name = df_name + 'LR2'
    print("[INFO] NAME OF CLF ",clf_name )
    clf_name = LogisticRegression()
    clf_name.fit(train[list_no_nan], train['category_3'])
    #save the model in pickel
    pickle.dump(clf_name, open('clf_name.sav', 'wb'))

    print("[INFO] Imputing Categorical 3 values ....")
    df.loc[df['category_3'].isna(), 'category_3'] = clf_name.predict(test[list_no_nan])

    df['card_id'] = temp['card_id']
    df['merchant_id'] = temp['merchant_id']
    df['purchase_date'] = temp['purchase_date']

    print(df.isna().sum())

    #FILL MISSING MERCHANT IDS 
    df['merchant_id'].fillna('M_ID_00a6ca8a8a', inplace=True)
    df = lab_enc(df,['category_3'], prefix=df_name)


    print(df.isna().sum())

    print("[INFO] SAVING ...")
    file_name = df_name
    df.to_csv('file_name.csv', index=False)


    return df


## Imputations on Merchant Data

In [41]:
def merchant_imputations(num_rows=None):
    """
    FUNCTION:
          PERFORMS THE IMPUTATIONS FOR THE TRANSACTIONS DATA 
          IT HANDLES MISSING VALUES 
          IT DOES MAPPING OF CATEGORICAL DATA TO NUMERICAL
          IT HANDLES THE INF VALUES  

    ARGS:
          NUM_ROWS: TO LOAD THE NUMBER OF ROWS FOR THE DATAFRAME(DEBUGGING)
          DEFAULT IS NONE
    
    RETURNS:
          THE MODIFIED DATA FRAME WITH IMPUTATIONS 

    REFER:-https://medium.com/towards-artificial-intelligence/handling-missing-data-for-advanced-machine-learning-b6eb89050357
    """

    import pickle
    from sklearn.linear_model import LogisticRegression
    from sklearn.neighbors import KNeighborsRegressor

    df = pd.read_csv('/content/drive/MyDrive/data/merchants.csv', nrows=num_rows)
    df = reduce_mem_usage(df)

    #get all the ones not inf
    #suprisingly the 3 values have the nan and inf
    df = df[df['avg_purchases_lag3']!=np.inf]

    #creating a temporary dataframe 
    temp = pd.DataFrame()
    temp['merchant_id'] = df['merchant_id']
    # we use KNN for numerical and logisticregrssion for categorical 
    temp['category_2'] = df['category_2']

    #category 2 is needed for 2nd round for logistic regression
    # merchant id is not useful
    df.drop(['merchant_id', 'category_2'], axis=1, inplace=True)

    #categorical to numerical 
    df = lab_enc(df, ['category_4','category_1','most_recent_sales_range','most_recent_purchases_range'], prefix='merchant')

    features = df.columns
    #sales have missing values 
    cols = ['avg_sales_lag3','avg_sales_lag6','avg_sales_lag12']
    list_no_nan = []

    #get the features not to predict and having no nan 
    for c in features:
      if c not in cols:
        list_no_nan.append(c)

    #have all nan as we need to predict this (avg_sales_lag3)
    test = df[df['avg_sales_lag3'].isna()]
    #have no nan to train 
    train = df.dropna()

    #initilize for knn 
    knn_sal_3 = KNeighborsRegressor(n_neighbors=5)

    #fit on data 
    knn_sal_3.fit(train[list_no_nan], train['avg_sales_lag3'])
    #fill the missing values 
    df.loc[df['avg_sales_lag3'].isna(), 'avg_sales_lag3'] = knn_sal_3.predict(test[list_no_nan])
    #save the model 
    pickle.dump(knn_sal_3, open('knn_sal_3.sav', 'wb'))

    #have all nan as we need to predict this (avg_sales_lag6)
    test = df[df['avg_sales_lag6'].isna()]
    train = df.dropna()

    #initilize the model
    knn_sal_6 = KNeighborsRegressor(n_neighbors=5)
    #fit the model 
    knn_sal_6.fit(train[list_no_nan], train['avg_sales_lag6'])
    #fill the missing values 
    df.loc[df['avg_sales_lag6'].isna(), 'avg_sales_lag6'] = knn_sal_6.predict(test[list_no_nan])
    #save the model 
    pickle.dump(knn_sal_6, open('knn_sal_6.sav', 'wb'))

    #have all nan as we need to predict this (avg_sales_lag12)
    test = df[df['avg_sales_lag12'].isna()]
    train = df.dropna()

    #initilize the model
    knn_sal_12 = KNeighborsRegressor(n_neighbors=5)
    #fit on data 
    knn_sal_12.fit(train[list_no_nan], train['avg_sales_lag12'])
    #fill the missing values 
    df.loc[df['avg_sales_lag12'].isna(), 'avg_sales_lag12'] = knn_sal_12.predict(test[list_no_nan])
    #save the model 
    pickle.dump(knn_sal_12, open('knn_sal_12.sav', 'wb'))

    #for category 2
    df['category_2'] = temp['category_2']

    feat = df.columns
    cols = ['category_2']
    list_no_nan = []

    #get the features not to predict and having no nan 
    for c in feat:
      if c not in cols:
        list_no_nan.append(c)

    #get all the missing values 
    test = df[df['category_2'].isna()]
    # have all the good values 
    train = df.dropna()

    LR_cat_2 = LogisticRegression()
    LR_cat_2.fit(train[list_no_nan], train['category_2'])
    df.loc[df['category_2'].isna(), 'category_2'] = LR_cat_2.predict(test[list_no_nan])
    pickle.dump(LR_cat_2, open('LR_cat_2.sav', 'wb'))

    df['merchant_id'] = temp['merchant_id']
    df.columns = [col+"_merchants_t" if col!="merchant_id" else col for col in df.columns]

    return df

## Basic FE and Grouping on card id 

In [42]:
def group_on_card_id_withMer1(df):
    """
    FUNCTION:
            TO GROUP THE DATA ON CARD ID 
            BASIC DATE AND TIME FEATURE ENGINEERING  
            THIS INCUDES MERCHANTS DATA TOO 

    ARGS:
            DF:- THE DATAFRAME ON WHICH THE BASIC FEATURE ENGINEERING AND 
                  GROUPING ON CARD ID IS DONE 

    RETURNS:
            THE MODIFIED DATA FRAME 
    
    """

    #removing purchase amount outliner    
    df['purchase_amount'] = df['purchase_amount'].apply(lambda x: min(x, 0.8))
    df['price'] = df['purchase_amount'] / (df['installments'] + 0.001) #some epsilone for 0  installments 

    #Some feature engineering on date and time  
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['month'] = df['purchase_date'].dt.month
    df['day'] = df['purchase_date'].dt.day
    df['hour'] = df['purchase_date'].dt.hour
    df['weekofyear'] = df['purchase_date'].dt.weekofyear
    df['weekday'] = df['purchase_date'].dt.weekday
    df['weekend'] = (df['purchase_date'].dt.weekday >= 5).astype(int)
    df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']


      
    def mode(series):
        """Most common element in a series"""
        tmode = series.mode()
        if len(tmode) == 0:
            return np.nan
        else:
            return tmode[0]

    agg_fun = {
      
    #CATEGORICAL VALUES  
        
    "authorized_flag": ['sum', 'mean',mode],
    'category_1' : ['sum', 'mean',mode],
    'category_2' :['sum', 'mean',mode],
    'category_3' :['sum', 'mean',mode],
    'category_1_merchants_t' :['sum', 'mean',mode],
    'most_recent_sales_range_merchants_t' :['sum', 'mean',mode],
    'category_4_merchants_t' :['sum', 'mean',mode],
    'most_recent_purchases_range_merchants_t' :['sum', 'mean',mode],
    'category_2_merchants_t' :['sum', 'mean',mode],


    # ID VALUES 
    'city_id' : ['nunique',mode],
    'state_id' : ['nunique',mode],
    'subsector_id' : ['nunique',mode],
    'merchant_category_id' : ['nunique',mode],
    'merchant_id': ['nunique',mode],
    'merchant_group_id_merchants_t':['nunique',mode],
    'state_id_merchants_t':['nunique',mode],
    'merchant_category_id_merchants_t':['nunique',mode],
    'subsector_id_merchants_t':['nunique',mode],
    'city_id_merchants_t':['nunique',mode],


    #NUMERICAL 
    'month_lag' : ['sum', 'mean', 'min', 'max', 'std','var'],
    'installments' : ['sum', 'mean', 'min', 'max', 'std','var'],
    'purchase_amount' : ['sum', 'mean', 'min', 'max', 'std','var'],
    'numerical_2_merchants_t': ['sum', 'mean', 'min', 'max', 'var', 'skew'],
    'avg_sales_lag3_merchants_t': ['sum', 'mean', 'min', 'max', 'var', 'skew'],
    'avg_purchases_lag3_merchants_t': ['sum', 'mean', 'min', 'max', 'var', 'skew'],
    'active_months_lag3_merchants_t': ['sum', 'mean', 'min', 'max', 'var', 'skew'],
    'avg_sales_lag6_merchants_t': ['sum', 'mean', 'min', 'max', 'var', 'skew'],
    'avg_purchases_lag6_merchants_t': ['sum', 'mean', 'min', 'max', 'var', 'skew'],
    'avg_sales_lag12_merchants_t': ['sum', 'mean', 'min', 'max', 'var', 'skew'],
    'avg_purchases_lag12_merchants_t': ['sum', 'mean', 'min', 'max', 'var', 'skew'],
    'numerical_1_merchants_t': ['sum', 'mean', 'min', 'max', 'var', 'skew'],
    'active_months_lag6_merchants_t': ['sum', 'mean', 'min', 'max', 'var', 'skew'],
    'active_months_lag12_merchants_t': ['sum', 'mean', 'min', 'max', 'var', 'skew'],
    'active_months_lag12_merchants_t': ['sum', 'mean', 'min', 'max', 'var', 'skew'],

    #DATE TIME 
    'weekend': ['nunique', 'mean', 'min', 'max'],
    'weekday' : ['nunique', 'mean', 'min', 'max'],
    'hour': ['nunique', 'mean', 'min', 'max'],
    'weekofyear': ['nunique', 'mean', 'min', 'max'],
    'day': ['nunique', 'mean', 'min', 'max'],
    #Refer:-https://numpy.org/doc/stable/reference/generated/numpy.ptp.html
    'purchase_date' : [np.ptp, 'min', 'max'],
    'month' : ['sum', 'mean', 'nunique']

    }

    df = df.groupby("card_id",as_index=False).agg(agg_fun)
    print(df.card_id)

    # reduce memory usage
    df = reduce_mem_usage(df)

    return df

In [43]:
def group_on_card_id_withOUTMer1(df):
    """
    FUNCTION:
            TO GROUP THE DATA ON CARD ID 
            BASIC DATE AND TIME FEATURE ENGINEERING  

    ARGS:
            DF:- THE DATAFRAME ON WHICH THE BASIC FEATURE ENGINEERING AND 
                  GROUPING ON CARD ID IS DONE 

    RETURNS:
            THE MODIFIED DATA FRAME 
    
    """

    #removing purchase amount outliner    
    df['purchase_amount'] = df['purchase_amount'].apply(lambda x: min(x, 0.8))
    df['price'] = df['purchase_amount'] / (df['installments'] + 0.001) #some epsilone for 0  installments 

    #Some feature engineering on date and time  
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['month'] = df['purchase_date'].dt.month
    df['day'] = df['purchase_date'].dt.day
    df['hour'] = df['purchase_date'].dt.hour
    df['weekofyear'] = df['purchase_date'].dt.weekofyear
    df['weekday'] = df['purchase_date'].dt.weekday
    df['weekend'] = (df['purchase_date'].dt.weekday >= 5).astype(int)
    df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']


      
    def mode(series):
        """Most common element in a series"""
        tmode = series.mode()
        if len(tmode) == 0:
            return np.nan
        else:
            return tmode[0]

    agg_fun = {
      
    #CATEGORICAL VALUES  
        
    "authorized_flag": ['sum', 'mean',mode],
    'category_1' : ['sum', 'mean',mode],
    'category_2' :['sum', 'mean',mode],
    'category_3' :['sum', 'mean',mode],
    #'category_1_merchants_t' :['sum', 'mean',mode],
    #'most_recent_sales_range_merchants_t' :['sum', 'mean',mode],
    #'category_4_merchants_t' :['sum', 'mean',mode],
    #'most_recent_purchases_range_merchants_t' :['sum', 'mean',mode],
    #'category_2_merchants_t' :['sum', 'mean',mode],


    # ID VALUES 
    'city_id' : ['nunique',mode],
    'state_id' : ['nunique',mode],
    'subsector_id' : ['nunique',mode],
    'merchant_category_id' : ['nunique',mode],
    'merchant_id': ['nunique',mode],
    #'merchant_group_id_merchants_t':['nunique',mode],
    #'state_id_merchants_t':['nunique',mode],
    #'merchant_category_id_merchants_t':['nunique',mode],
    #'subsector_id_merchants_t':['nunique',mode],
    #'city_id_merchants_t':['nunique',mode],


    #NUMERICAL 
    'month_lag' : ['sum', 'mean', 'min', 'max', 'std','var'],
    'installments' : ['sum', 'mean', 'min', 'max', 'std','var'],
    'purchase_amount' : ['sum', 'mean', 'min', 'max', 'std','var'],
    #'numerical_2_merchants_t': ['sum', 'mean', 'min', 'max', 'var', 'skew'],
    #'avg_sales_lag3_merchants_t': ['sum', 'mean', 'min', 'max', 'var', 'skew'],
    #'avg_purchases_lag3_merchants_t': ['sum', 'mean', 'min', 'max', 'var', 'skew'],
    #'active_months_lag3_merchants_t': ['sum', 'mean', 'min', 'max', 'var', 'skew'],
    #'avg_sales_lag6_merchants_t': ['sum', 'mean', 'min', 'max', 'var', 'skew'],
    #'avg_purchases_lag6_merchants_t': ['sum', 'mean', 'min', 'max', 'var', 'skew'],
    #'avg_sales_lag12_merchants_t': ['sum', 'mean', 'min', 'max', 'var', 'skew'],
    #'avg_purchases_lag12_merchants_t': ['sum', 'mean', 'min', 'max', 'var', 'skew'],
    #'numerical_1_merchants_t': ['sum', 'mean', 'min', 'max', 'var', 'skew'],
    #'active_months_lag6_merchants_t': ['sum', 'mean', 'min', 'max', 'var', 'skew'],
    #'active_months_lag12_merchants_t': ['sum', 'mean', 'min', 'max', 'var', 'skew'],
    #'active_months_lag12_merchants_t': ['sum', 'mean', 'min', 'max', 'var', 'skew'],

    #DATE TIME 
    'weekend': ['nunique', 'mean', 'min', 'max'],
    'weekday' : ['nunique', 'mean', 'min', 'max'],
    'hour': ['nunique', 'mean', 'min', 'max'],
    'weekofyear': ['nunique', 'mean', 'min', 'max'],
    'day': ['nunique', 'mean', 'min', 'max'],
    #Refer:-https://numpy.org/doc/stable/reference/generated/numpy.ptp.html
    'purchase_date' : [np.ptp, 'min', 'max'],
    'month' : ['sum', 'mean', 'nunique']

    }

    df = df.groupby("card_id",as_index=False).agg(agg_fun)
    print(df.card_id)

    # reduce memory usage
    df = reduce_mem_usage(df)

    return df

## Generating the aurgumentated dataset 

In [44]:
def imputations_merAndtrans(debug=False):

    """ 
    FUNCTION:
      Do model based imputation 
      STEPS:-
      1. Load and Impute Historical Transactions
      2. Load and Impute New Merchants Transactions
      3. Load and Impute  Merchants   

    ARGS:
     Debug:- Takes care of numer of rows as we need faster manipulation of data in dubigging 

    RETURNS:
      Generates the new_merchant_df, historical_transactions_df, merchant_df 

    """

    num_rows = 10000 if debug else None
    with timer("load and impute transactions ...."):
        # load AND IMPUTE 
        print("[INFO] LOADING AND IMPUTING HISTORICAL TRANSACTIONS ....")
        historical_transactions_df = transactions_imputations(df_name='hist',nrows = num_rows)
        print("[INFO] LOADING AND IMPUTING HISTORICAL TRANSACTIONS ....")
        new_merchant_df = transactions_imputations(df_name='mer',nrows = num_rows)

        #unique values of column of card id 
        print("[INFO] THE UNIQUE VALUES OF CARD ID ...")
        print("FOR NEW MERCHANT ",new_merchant_df.card_id.nunique())
        print("FOR HISTORICAL TRANSACTIONS ",historical_transactions_df.card_id.nunique())
        print("[INFO] Check for Nan")
        print(historical_transactions_df.isna().sum())
        print(historical_transactions_df.isna().sum())

    with timer("load and impute merchants ...."):
        #load csv 
        print("[INFO] LOADING MERCHANTS AND IMPUTATIONS  ....")
        merchant_df = merchant_imputations(num_rows=num_rows)

        print("[INFO] Check for Nan ...")
        print(merchant_df.isna().sum())    

        return new_merchant_df, historical_transactions_df, merchant_df

In [50]:
def basicFeAndGrouping(new_merchant_df, historical_transactions_df, merchant_df):

    """ 
    FUNCTION:
      Perform merging of data, grouping and basic feature engineering 
      STEPS:-
      1. 
      2. 
      3.    

    ARGS:
     Debug:- Takes care of numer of rows as we need faster manipulation of data in dubigging 

    RETURNS:
      Generates the aurgumented train and test 

    """

    with timer("Merging Data frames ...."):
        #Merge the data frames 
        print("[INFO] MERGING THE DATA FRAMES ...")
        historical_transactions_df = pd.merge(historical_transactions_df, merchant_df, on='merchant_id', how = 'left')
        new_merchant_df = pd.merge(new_merchant_df, merchant_df, on='merchant_id',how = 'left')

        #DEL merchant_df
        del merchant_df

        print("[INFO] THE UNIQUE VALUES OF CARD ID ...")
        #unique values of column of card id 
        print("FOR NEW MERCHANT ",new_merchant_df.card_id.nunique())
        print("FOR HISTORICAL TRANSACTIONS ",historical_transactions_df.card_id.nunique())

        print("[INFO] THE COLUMNS  ...")
        print("FOR NEW MERCHANT ",new_merchant_df.columns)
        print("FOR HISTORICAL TRANSACTIONS ",historical_transactions_df.columns)


    with timer("group on card id ..."):
        #Group on card id
        print("[INFO] GROUPING ON CARD ID ...")
        new_merchant_df = group_on_card_id_withMer1(new_merchant_df)
        historical_transactions_df = group_on_card_id_withMer1(historical_transactions_df)


        print("[INFO] THE UNIQUE VALUES OF CARD ID ...")
        #unique values of column of card id 
        print("FOR NEW MERCHANT ",new_merchant_df.card_id.nunique())
        print("FOR HISTORICAL TRANSACTIONS ",historical_transactions_df.card_id.nunique())
    with timer("Changing column names ...."):
        print("[INFO] Changing column names ....")
        historical_transactions_df.columns = ["_".join(col) for col in historical_transactions_df.columns.ravel()]
        historical_transactions_df.rename(columns={'card_id_':'card_id'},inplace=True)
        new_merchant_df.columns = ["_".join(col) for col in new_merchant_df.columns.ravel()]
        new_merchant_df.rename(columns={'card_id_':'card_id'},inplace=True)
    with timer("Train test"):
        df = train_test(None)
        print(df)

    with timer("merge hist and new merchant with  df  "):
        print("[INFO] MERGING ....")
        df = pd.merge(df, historical_transactions_df, on='card_id', how='outer')
        print(df)
        df = pd.merge(df, new_merchant_df, on='card_id', how='outer')
        print(df)
        

    with timer("split train & test"):
        print("[INFO] Split train and test ...")
        train_df = df[df['target'].notnull()]
        test_df = df[df['target'].isnull()]

        del test_df['target']
        del df
        gc.collect()

    with timer("Save train and test files"):
        print("[INFO] SAVING ....")
        train_df.to_csv('augmented_train_try1_model_imputation.csv', index=False)
        test_df.to_csv('augmented_test_try1_model_imputation.csv', index=False)
        return train_df,test_df
    

In [51]:
def basicFeAndGroupingwithoutMer(new_merchant_df, historical_transactions_df):

    """ 
    FUNCTION:
      Perform merging of data, grouping and basic feature engineering 
      STEPS:-
      1. 
      2. 
      3.    

    ARGS:
     Debug:- Takes care of numer of rows as we need faster manipulation of data in dubigging 

    RETURNS:
      Generates the aurgumented train and test 

    """
    with timer("group on card id ..."):
        #Group on card id
        print("[INFO] GROUPING ON CARD ID ...")
        new_merchant_df = group_on_card_id_withOUTMer1(new_merchant_df)
        historical_transactions_df = group_on_card_id_withOUTMer1(historical_transactions_df)


        print("[INFO] THE UNIQUE VALUES OF CARD ID ...")
        #unique values of column of card id 
        print("FOR NEW MERCHANT ",new_merchant_df.card_id.nunique())
        print("FOR HISTORICAL TRANSACTIONS ",historical_transactions_df.card_id.nunique())

    with timer("Changing column names ...."):
        print("[INFO] Changing column names ....")
        historical_transactions_df.columns = ["_".join(col) for col in historical_transactions_df.columns.ravel()]
        historical_transactions_df.rename(columns={'card_id_':'card_id'},inplace=True)
        new_merchant_df.columns = ["_".join(col) for col in new_merchant_df.columns.ravel()]
        new_merchant_df.rename(columns={'card_id_':'card_id'},inplace=True)
        
    with timer("Train test"):
        df = train_test(None)
        print(df)

    with timer("merge hist and new merchant with  df  "):
        print("[INFO] MERGING ....")
        df = pd.merge(df, historical_transactions_df, on='card_id', how='outer')
        print(df)
        df = pd.merge(df, new_merchant_df, on='card_id', how='outer')
        print(df)
        

    with timer("split train & test"):
        print("[INFO] Split train and test ...")
        train_df = df[df['target'].notnull()]
        test_df = df[df['target'].isnull()]

        del test_df['target']
        del df
        gc.collect()

    with timer("Save train and test files"):
        print("[INFO] SAVING ....")
        train_df.to_csv('augmented_train_try1_withoutMer.csv', index=False)
        test_df.to_csv('augmented_test_try1_withoutMer.csv', index=False)
        return train_df,test_df

## Main Method

In [66]:
if __name__ == "__main__":
  new_merchant_df,historical_transactions_df,merchant_df = imputations_merAndtrans(debug=False)
  train_df,test_df = basicFeAndGrouping(new_merchant_df, historical_transactions_df, merchant_df)

[INFO] LOADING AND IMPUTING HISTORICAL TRANSACTIONS ....
Mem. usage decreased to 1749.11 Mb (43.7% reduction)
authorized_flag               0
card_id                       0
city_id                       0
category_1                    0
installments                  0
category_3               178159
merchant_category_id          0
merchant_id              138481
month_lag                     0
purchase_amount               0
purchase_date                 0
category_2              2652864
state_id                      0
subsector_id                  0
dtype: int64
          authorized_flag  city_id  ...  state_id  subsector_id
17615289                1       69  ...         9             7
22074357                1      158  ...        15            34
27795103                1       78  ...         9            27
17432676                1       69  ...         9            34
28874760                1       69  ...         9            37
21007181                1      118  ...      

In [67]:
!ls

 augmented_test.csv
 augmented_test_try1.csv
 augmented_test_try1_model_imputation.csv
 augmented_train.csv
 augmented_train_try1.csv
 augmented_train_try1_model_imputation.csv
 clf_cat2.sav
 clf_cat3.sav
 clf_name.sav
'Data Dictionary.xlsx'
 Data_Dictionary.xlsx
 file_name.csv
 hist_authorized_flag_enc.npy
 hist_category_1_enc.npy
 hist_category_3_enc.npy
 historical_transactions.csv
 historical_transactions_df.pkl
 knn_sal_12.sav
 knn_sal_3.sav
 knn_sal_6.sav
 LR_cat_2.sav
 mer_authorized_flag_enc.npy
 mer_category_1_enc.npy
 mer_category_3_enc.npy
 merchant_category_1_enc.npy
 merchant_category_4_enc.npy
 merchant_most_recent_purchases_range_enc.npy
 merchant_most_recent_sales_range_enc.npy
 merchants.csv
 merged_transactions_with_merchants.csv
 new_merchant_authorized_flag_enc.npy
 new_merchant_category_1_enc.npy
 new_merchant_df.pkl
 new_merchant_transactions.csv
 sample_submission.csv
 test.csv
 train.csv
 transactions_raw_merged.csv
 transactions_refined_1_merged_WON.csv


In [68]:
new_merchant_df

,authorized_flag,city_id,category_1,installments,category_3,merchant_category_id,month_lag,purchase_amount,category_2,state_id,subsector_id,card_id,merchant_id,purchase_date
0,0,107,0,1,1,307,1,-0.557617,1.0,9,19,C_ID_415bb3a509,M_ID_b0c793002c,2018-03-11 14:57:36
1,0,140,0,1,1,307,1,-0.569336,1.0,9,19,C_ID_415bb3a509,M_ID_88920c89e8,2018-03-19 18:53:37
2,0,330,0,1,1,507,2,-0.551270,1.0,9,14,C_ID_415bb3a509,M_ID_ad5237ef6b,2018-04-26 14:08:44
3,0,-1,1,1,1,661,1,-0.671875,1.0,-1,8,C_ID_415bb3a509,M_ID_9e84cda3b1,2018-03-07 09:43:21
4,0,-1,1,1,1,166,1,-0.659668,1.0,-1,29,C_ID_ef55cf8d4b,M_ID_3c86fa3831,2018-03-22 21:07:53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1963026,0,142,0,0,0,309,2,-0.701660,3.0,19,21,C_ID_1320dee851,M_ID_7754b67f3b,2018-04-06 14:36:52
1963027,0,158,0,0,0,560,2,-0.694336,1.0,15,34,C_ID_f112aa3381,M_ID_da063195b7,2018-03-07 13:19:18
1963028,0,69,0,1,1,278,1,-0.621094,1.0,9,37,C_ID_bd97b86450,M_ID_9a9ccb6544,2018-03-05 12:04:56
1963029,0,130,0,0,0,367,1,-0.656738,3.0,7,16,C_ID_c0513fd84f,M_ID_40c28d596f,2018-03-09 14:47:05


In [69]:
historical_transactions_df

,authorized_flag,city_id,category_1,installments,category_3,merchant_category_id,month_lag,purchase_amount,category_2,state_id,subsector_id,card_id,merchant_id,purchase_date
0,1,88,0,0,0,80,-8,-0.703331,1.0,16,37,C_ID_4e6213e9bc,M_ID_e020e9b302,2017-06-25 15:33:07
1,1,88,0,0,0,367,-7,-0.733128,1.0,16,16,C_ID_4e6213e9bc,M_ID_86ec983688,2017-07-15 12:10:45
2,1,88,0,0,0,80,-6,-0.720386,1.0,16,37,C_ID_4e6213e9bc,M_ID_979ed661fc,2017-08-09 22:04:29
3,1,88,0,0,0,560,-5,-0.735352,1.0,16,34,C_ID_4e6213e9bc,M_ID_e6d5ae8ea6,2017-09-02 10:06:26
4,1,88,0,0,0,80,-11,-0.722865,1.0,16,37,C_ID_4e6213e9bc,M_ID_e020e9b302,2017-03-10 01:14:19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29112356,1,-1,1,1,1,360,-1,-0.632706,1.0,-1,34,C_ID_2863d2fa95,M_ID_edd92b6720,2017-01-20 08:52:04
29112357,1,-1,1,1,1,360,0,-0.632706,1.0,-1,34,C_ID_2863d2fa95,M_ID_edd92b6720,2017-02-20 04:40:50
29112358,1,3,0,0,0,278,0,-0.657740,1.0,16,37,C_ID_5c240d6e3c,M_ID_9cdcfe8673,2017-12-26 18:37:51
29112359,1,331,0,0,0,514,-1,0.770620,1.0,16,9,C_ID_5c240d6e3c,M_ID_1a75f94f92,2017-11-24 14:18:15


In [70]:
merchant_df


,merchant_group_id_merchants_t,merchant_category_id_merchants_t,subsector_id_merchants_t,numerical_1_merchants_t,numerical_2_merchants_t,category_1_merchants_t,most_recent_sales_range_merchants_t,most_recent_purchases_range_merchants_t,avg_sales_lag3_merchants_t,avg_purchases_lag3_merchants_t,active_months_lag3_merchants_t,avg_sales_lag6_merchants_t,avg_purchases_lag6_merchants_t,active_months_lag6_merchants_t,avg_sales_lag12_merchants_t,avg_purchases_lag12_merchants_t,active_months_lag12_merchants_t,category_4_merchants_t,city_id_merchants_t,state_id_merchants_t,category_2_merchants_t,merchant_id
0,8353,792,9,-0.057465,-0.057465,0,4,4,-0.400000,9.666667,3,-2.250000,18.666667,6,-2.320000,13.916667,12,0,242,9,1.0,M_ID_838061e48c
1,3184,840,20,-0.057465,-0.057465,0,4,4,-0.720000,1.750000,3,-0.740000,1.291667,6,-0.570000,1.687500,12,0,22,16,1.0,M_ID_9339d880ad
2,447,690,1,-0.057465,-0.057465,0,4,4,-82.129997,260.000000,2,-82.129997,260.000000,2,-82.129997,260.000000,2,0,-1,5,5.0,M_ID_e726bbae1e
3,5026,792,9,-0.057465,-0.057465,1,4,4,1.244000,1.666667,3,2.660000,4.666667,6,5.232000,3.833333,12,1,-1,-1,1.0,M_ID_a70e9c5f81
4,2228,222,21,-0.057465,-0.057465,1,4,4,4.280000,0.500000,3,3.308000,0.361111,6,3.466000,0.347222,12,1,-1,-1,1.0,M_ID_64456c37ce
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334691,1145,705,33,3.173828,-0.047546,0,0,0,1.000000,1.022618,3,0.990000,1.019505,6,1.000000,1.024796,12,1,69,9,1.0,M_ID_1f4773aa76
334692,35,544,29,-0.057465,-0.057465,1,0,0,0.890000,0.927684,3,0.780000,0.813473,6,0.590000,0.606765,12,1,-1,-1,1.0,M_ID_725a60d404
334693,35,561,7,-0.057465,-0.057465,0,0,0,0.960000,0.982957,3,0.900000,0.924769,6,0.740000,0.750763,8,1,160,21,5.0,M_ID_f2045dd267
334694,35,511,7,-0.057465,-0.057465,1,0,0,0.940000,0.919558,3,0.820000,0.783000,6,0.650000,0.584000,12,1,-1,-1,1.0,M_ID_9139332ccc


In [71]:
new_merchant_df.isna().sum()

authorized_flag         0
city_id                 0
category_1              0
installments            0
category_3              0
merchant_category_id    0
month_lag               0
purchase_amount         0
category_2              0
state_id                0
subsector_id            0
card_id                 0
merchant_id             0
purchase_date           0
dtype: int64

In [72]:
historical_transactions_df.isna().sum()

authorized_flag         0
city_id                 0
category_1              0
installments            0
category_3              0
merchant_category_id    0
month_lag               0
purchase_amount         0
category_2              0
state_id                0
subsector_id            0
card_id                 0
merchant_id             0
purchase_date           0
dtype: int64

In [73]:
merchant_df.isna().sum()

merchant_group_id_merchants_t              0
merchant_category_id_merchants_t           0
subsector_id_merchants_t                   0
numerical_1_merchants_t                    0
numerical_2_merchants_t                    0
category_1_merchants_t                     0
most_recent_sales_range_merchants_t        0
most_recent_purchases_range_merchants_t    0
avg_sales_lag3_merchants_t                 0
avg_purchases_lag3_merchants_t             0
active_months_lag3_merchants_t             0
avg_sales_lag6_merchants_t                 0
avg_purchases_lag6_merchants_t             0
active_months_lag6_merchants_t             0
avg_sales_lag12_merchants_t                0
avg_purchases_lag12_merchants_t            0
active_months_lag12_merchants_t            0
category_4_merchants_t                     0
city_id_merchants_t                        0
state_id_merchants_t                       0
category_2_merchants_t                     0
merchant_id                                0
dtype: int

In [74]:
train_df

,card_id,first_active_month,feature_1,feature_2,feature_3,target,outliers,quarter,elapsed_time,quarter_first_active_month,first_active_month_diff_from_today,feature_sum,feature_mean,feature_max,feature_min,feature_std,authorized_flag_sum_x,authorized_flag_mean_x,authorized_flag_mode_x,category_1_sum_x,category_1_mean_x,category_1_mode_x,category_2_sum_x,category_2_mean_x,category_2_mode_x,category_3_sum_x,category_3_mean_x,category_3_mode_x,category_1_merchants_t_sum_x,category_1_merchants_t_mean_x,category_1_merchants_t_mode_x,most_recent_sales_range_merchants_t_sum_x,most_recent_sales_range_merchants_t_mean_x,most_recent_sales_range_merchants_t_mode_x,category_4_merchants_t_sum_x,category_4_merchants_t_mean_x,category_4_merchants_t_mode_x,most_recent_purchases_range_merchants_t_sum_x,most_recent_purchases_range_merchants_t_mean_x,most_recent_purchases_range_merchants_t_mode_x,...,numerical_1_merchants_t_var_y,numerical_1_merchants_t_skew_y,active_months_lag6_merchants_t_sum_y,active_months_lag6_merchants_t_mean_y,active_months_lag6_merchants_t_min_y,active_months_lag6_merchants_t_max_y,active_months_lag6_merchants_t_var_y,active_months_lag6_merchants_t_skew_y,active_months_lag12_merchants_t_sum_y,active_months_lag12_merchants_t_mean_y,active_months_lag12_merchants_t_min_y,active_months_lag12_merchants_t_max_y,active_months_lag12_merchants_t_var_y,active_months_lag12_merchants_t_skew_y,weekend_nunique_y,weekend_mean_y,weekend_min_y,weekend_max_y,weekday_nunique_y,weekday_mean_y,weekday_min_y,weekday_max_y,hour_nunique_y,hour_mean_y,hour_min_y,hour_max_y,weekofyear_nunique_y,weekofyear_mean_y,weekofyear_min_y,weekofyear_max_y,day_nunique_y,day_mean_y,day_min_y,day_max_y,purchase_date_ptp_y,purchase_date_min_y,purchase_date_max_y,month_sum_y,month_mean_y,month_nunique_y
0,C_ID_92a2005557,2017-06-01,1.0,1.0,1.0,-0.820283,1.0,2.0,1310.0,2.0,1310.0,3.0,1.0,1.0,1.0,0.0,257,0.948242,1,0,0.000000,0,283.0,1.043945,1.0,4,0.014763,0,36.0,0.132812,0.0,451.0,1.664062,0.0,259.0,0.955566,1.0,390.0,1.439453,0.0,...,3156.000000,inf,137.0,5.957031,5.0,6.0,0.043488,-4.796875,267.0,11.609375,5.0,12.0,2.248047,-4.304688,2.0,0.260986,0.0,1.0,7.0,3.130859,0.0,6.0,8.0,12.867188,8.0,16.0,7.0,13.304688,10.0,17.0,17.0,16.437500,5.0,31.0,54 days 21:18:29,2018-03-05 14:04:36,2018-04-29 11:23:05,80.0,3.478516,2.0
1,C_ID_3d0044924f,2017-01-01,1.0,1.0,1.0,0.392913,1.0,1.0,1461.0,1.0,1461.0,3.0,1.0,1.0,1.0,0.0,354,0.969727,1,35,0.095886,0,365.0,1.000000,1.0,442,1.210938,1,62.0,0.169922,0.0,392.0,1.074219,0.0,336.0,0.920410,1.0,389.0,1.065430,0.0,...,1.946289,2.447266,36.0,6.000000,6.0,6.0,0.000000,0.000000,72.0,12.000000,12.0,12.0,0.000000,0.000000,1.0,0.000000,0.0,0.0,4.0,1.500000,0.0,4.0,5.0,11.164062,6.0,17.0,4.0,9.000000,5.0,13.0,4.0,13.500000,1.0,30.0,56 days 13:40:32,2018-02-01 17:07:54,2018-03-30 06:48:26,15.0,2.500000,2.0
2,C_ID_d639edf6cd,2016-08-01,1.0,1.0,1.0,0.688056,1.0,3.0,1614.0,3.0,1614.0,3.0,1.0,1.0,1.0,0.0,42,0.954590,1,0,0.000000,0,200.0,4.546875,5.0,0,0.000000,0,2.0,0.045441,0.0,16.0,0.363525,0.0,41.0,0.931641,1.0,16.0,0.363525,0.0,...,NaN,NaN,6.0,6.000000,6.0,6.0,NaN,NaN,10.0,10.000000,10.0,10.0,NaN,NaN,1.0,1.000000,1.0,1.0,1.0,5.000000,5.0,5.0,1.0,17.000000,17.0,17.0,1.0,17.000000,17.0,17.0,1.0,28.000000,28.0,28.0,0 days 00:00:00,2018-04-28 17:43:11,2018-04-28 17:43:11,4.0,4.000000,1.0
3,C_ID_186d6a6901,2017-09-01,1.0,1.0,1.0,0.142495,1.0,3.0,1218.0,3.0,1218.0,3.0,1.0,1.0,1.0,0.0,89,1.000000,1,13,0.146118,0,257.0,2.886719,4.0,95,1.067383,1,30.0,0.337158,0.0,143.0,1.606445,0.0,87.0,0.977539,1.0,142.0,1.595703,0.0,...,0.112488,1.932617,42.0,6.000000,6.0,6.0,0.000000,0.000000,84.0,12.000000,12.0,12.0,0.000000,0.000000,2.0,0.428467,0.0,1.0,4.0,3.285156,1.0,6.0,5.0,13.000000,7.0,21.0,5.0,13.859375,10.0,16.0,7.0,13.140625,4.0,24.0,41 days 23:05:05,2018-03-07 11:55:06,2018-04-18 11:00:11,26.0,3.714844,2.0
4,C_ID_cdbd2c0db2,2017-11-01,1.0,1.0,1.0,-0.159749,1.0,4.0,1157.0,4.0,1157.0,3.0,1.0,1.0,1.0,0.0,138,0.964844,1,16,0.111877,0,474.0,3.314453

In [75]:
test_df

,card_id,first_active_month,feature_1,feature_2,feature_3,outliers,quarter,elapsed_time,quarter_first_active_month,first_active_month_diff_from_today,feature_sum,feature_mean,feature_max,feature_min,feature_std,authorized_flag_sum_x,authorized_flag_mean_x,authorized_flag_mode_x,category_1_sum_x,category_1_mean_x,category_1_mode_x,category_2_sum_x,category_2_mean_x,category_2_mode_x,category_3_sum_x,category_3_mean_x,category_3_mode_x,category_1_merchants_t_sum_x,category_1_merchants_t_mean_x,category_1_merchants_t_mode_x,most_recent_sales_range_merchants_t_sum_x,most_recent_sales_range_merchants_t_mean_x,most_recent_sales_range_merchants_t_mode_x,category_4_merchants_t_sum_x,category_4_merchants_t_mean_x,category_4_merchants_t_mode_x,most_recent_purchases_range_merchants_t_sum_x,most_recent_purchases_range_merchants_t_mean_x,most_recent_purchases_range_merchants_t_mode_x,category_2_merchants_t_sum_x,...,numerical_1_merchants_t_var_y,numerical_1_merchants_t_skew_y,active_months_lag6_merchants_t_sum_y,active_months_lag6_merchants_t_mean_y,active_months_lag6_merchants_t_min_y,active_months_lag6_merchants_t_max_y,active_months_lag6_merchants_t_var_y,active_months_lag6_merchants_t_skew_y,active_months_lag12_merchants_t_sum_y,active_months_lag12_merchants_t_mean_y,active_months_lag12_merchants_t_min_y,active_months_lag12_merchants_t_max_y,active_months_lag12_merchants_t_var_y,active_months_lag12_merchants_t_skew_y,weekend_nunique_y,weekend_mean_y,weekend_min_y,weekend_max_y,weekday_nunique_y,weekday_mean_y,weekday_min_y,weekday_max_y,hour_nunique_y,hour_mean_y,hour_min_y,hour_max_y,weekofyear_nunique_y,weekofyear_mean_y,weekofyear_min_y,weekofyear_max_y,day_nunique_y,day_mean_y,day_min_y,day_max_y,purchase_date_ptp_y,purchase_date_min_y,purchase_date_max_y,month_sum_y,month_mean_y,month_nunique_y
201917,C_ID_0ab67a22ab,2017-04-01,1.0,1.0,1.0,NaN,2.0,1371.0,2.0,1371.0,3.0,1.0,1.0,1.0,0.0,47,0.662109,1,23,0.323975,0,71.0,1.000000,1.0,93,1.309570,1,29.0,0.408447,0.0,116.0,1.633789,0.0,35.0,0.492920,0.0,118.0,1.662109,0.0,71.0,...,0.000131,1.730469,18.0,6.0,6.0,6.0,0.0,0.0,36.0,12.000000,12.0,12.0,0.000000,0.000000,2.0,0.333252,0.0,1.0,3.0,3.666016,2.0,5.0,3.0,13.664062,9.0,19.0,3.0,7.000000,5.0,9.0,3.0,15.664062,3.0,28.0,25 days 09:24:23,2018-02-03 09:44:29,2018-02-28 19:08:52,6.0,2.000000,1.0
201918,C_ID_130fd0cbdd,2017-01-01,1.0,1.0,1.0,NaN,1.0,1461.0,1.0,1461.0,3.0,1.0,1.0,1.0,0.0,77,0.987305,1,2,0.025635,0,305.0,3.910156,4.0,80,1.025391,1,2.0,0.025635,0.0,130.0,1.666992,2.0,3.0,0.038452,0.0,127.0,1.627930,2.0,306.0,...,1.009766,3.162109,60.0,6.0,6.0,6.0,0.0,0.0,111.0,11.101562,7.0,12.0,3.656250,-1.845703,2.0,0.300049,0.0,1.0,6.0,2.699219,0.0,6.0,8.0,15.703125,9.0,23.0,6.0,12.296875,9.0,16.0,7.0,11.398438,3.0,20.0,48 days 05:41:29,2018-03-03 12:18:48,2018-04-20 18:00:17,34.0,3.400391,2.0
201919,C_ID_b709037bc5,2017-08-01,1.0,1.0,1.0,NaN,3.0,1249.0,3.0,1249.0,3.0,1.0,1.0,1.0,0.0,9,0.692383,1,1,0.076904,0,57.0,4.382812,5.0,19,1.461914,2,1.0,0.076904,0.0,25.0,1.922852,2.0,11.0,0.846191,1.0,35.0,2.691406,3.0,61.0,...,0.053528,NaN,12.0,6.0,6.0,6.0,0.0,NaN,24.0,12.000000,12.0,12.0,0.000000,NaN,1.0,0.000000,0.0,0.0,2.0,2.000000,1.0,3.0,2.0,13.500000,13.0,14.0,2.0,10.000000,9.0,11.0,2.0,7.000000,1.0,13.0,11 days 22:19:13,2018-03-01 14:51:33,2018-03-13 13:10:46,6.0,3.000000,1.0
201920,C_ID_d27d835a9f,2017-12-01,1.0,1.0,1.0,NaN,4.0,1127.0,4.0,1127.0,3.0,1.0,1.0,1.0,0.0,26,1.000000,1,0,0.000000,0,26.0,1.000000,1.0,31,1.192383,1,0.0,0.000000,0.0,35.0,1.345703,2.0,26.0,1.000000,1.0,36.0,1.384766,0.0,26.0,...,0.458496,2.052734,60.0,6.0,6.0,6.0,0.0,0.0,120.0,12.000000,12.0,12.0,0.000000,0.000000,2.0,0.300049,0.0,1.0,5.0,3.199219,1.0,6.0,7.0,18.203125,11.0,21.0,7.0,12.101562,9.0,16.0,8.0,13.601562,4.0,31.0,44 days 10:15:54,2018-03-04 11:06:29,2018-04-17 21:22:23,33.0,3.300781,2.0
201921,C_ID_2b5e3df5c2,2015-12-01,1.0,1.0,1.0,NaN,4.0,1858.0,4.0,1858.0,3.0,1.0,1.0,1.0,0.0,90,0.796387,1,0,0.000000,0,426.0,3.769531,4.0,120,1.061523,1,6.0,0.053101,0.0,2

In [76]:
train_df.isnull().sum(axis = 0)

card_id                    0
first_active_month         0
feature_1                  0
feature_2                  0
feature_3                  0
                       ...  
purchase_date_min_y    21931
purchase_date_max_y    21931
month_sum_y            21931
month_mean_y           21931
month_nunique_y        21931
Length: 330, dtype: int64

In [77]:
#Gives the name of all the variables with missing data 
columns_with_na = [var for var in train_df.columns if train_df[var].isnull().mean()  > 0]

In [78]:
len(columns_with_na)

168